### ISSM CA3

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import glob
import time

### Dataset attributes
Column 1: Timestamp in seconds  
Column 2: Timestamp in microseconds  
Column 3-15: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S1  
Column 16-28: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S2  
Column 29-41: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S3  
Column 42-54: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S4  
Column 55-67: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S5  
Column 68-80: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S6  
Column 91-93: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S7  
Column 94-106: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S8  
Column 107-119: [AccX, AccY, AccZ, GyrX, GyrY, Gyr, GyrZ, MagX, MagY, MagZ, Q1, Q2, Q3, Q4] of sensor S9  
Column 120: Label (see activity set) [Walking = 1] [Jogging = 2] [Running = 3]

### Run following cells for the following dataset structure:
    *df_walking : all 17 subjects + walking + (accel + gyro + quart)
    *df_jogging : all 17 subjects + jogging + (accel + gyro + quart)
    *df_running : all 17 subjects + running + (accel + gyro + quart)
    
Col[0]: Timestamp in seconds  
Col[1]: Timestamp in microseconds  
Col[93-105]: Sensor S8  
Col[106-118]: Sensor S9  
Col[119]: Activity Label

In [99]:
data_dir = '..\dataset'

def readcsvDataset(directory):
    files = glob.glob(directory + '\*.csv')
    df_walk = []
    df_jog = []
    df_run = []
    for file in files:
        df_all = pd.read_csv(file, header=None, delim_whitespace=True)
        df_walk.append(df_all[df_all[119] == 1])
        df_jog.append(df_all[df_all[119] == 2])
        df_run.append(df_all[df_all[119] == 3])
    return pd.concat(df_walk, ignore_index=True), pd.concat(df_jog, ignore_index=True), pd.concat(df_run, ignore_index=True)

def removeOtherSensor(df_walk, df_jog, df_run):
    idx = np.r_[2:93]
    return df_walk.drop(df_walk.columns[idx], axis=1), df_jog.drop(df_jog.columns[idx], axis=1), df_run.drop(df_run.columns[idx], axis=1)

In [100]:
start = time.time()
dfw, dfj, dfr = readcsvDataset(data_dir)
df_walk, df_jog, df_run = removeOtherSensor(dfw, dfj, dfr)
end = time.time()
print(f'Elapsed: {str(end-start)}')

Elapsed: 32.123751640319824


In [101]:
df_walk.tail()

,0,1,93,94,95,96,97,98,99,100,...,110,111,112,113,114,115,116,117,118,119
45633,456,940000.0,-6.3261,0.49173,-2.06260,-0.69493,-0.325470,0.427830,0.59707,-0.081856,...,0.47527,0.022887,0.61756,-0.27884,0.095882,-0.56782,0.41634,-0.63496,-0.31793,1
45634,456,960000.0,-5.6887,0.27611,-1.49380,-0.73027,-0.089975,0.255660,0.59280,-0.082832,...,0.47707,0.010613,0.61893,-0.27946,0.101300,-0.56451,0.41788,-0.63780,-0.31610,1
45635,456,980000.0,-5.3004,0.17579,-0.65768,-0.79647,0.162780,0.167450,0.59254,-0.084551,...,0.49299,0.001066,0.62184,-0.27693,0.108670,-0.56105,0.41981,-0.64029,-0.31465,1
45636,457,0.0,-5.3568,0.35077,0.22515,-0.88075,0.397370,0.090353,0.59128,-0.086545,...,0.54279,-0.006686,0.62217,-0.27413,0.112640,-0.55720,0.42186,-0.64327,-0.31267,1
45637,457,20000.0,-5.4664,0.49299,1.02270,-0.80796,0.585730,0.030970,0.59328,-0.091122,...,0.55324,0.026911,0.62150,-0.27479,0.119120,-0.55344,0.42354,-0.64632,-0.31081,1


In [94]:
df_walk.shape

(45638, 28)

In [95]:
df_jog.shape

(41790, 28)

In [96]:
df_run.shape

(40142, 28)